In [1]:
from setup import setup_notebook
setup_notebook()
from pywhy import instrument_code, exec_instrumented

In [2]:
source_code = """
def add(a, b):

    if a < 10:
        c = a + b
        return c

    return b

a = 5
b = 10
add(a, b)

"""

insturmented_code = instrument_code(source_code)
print(insturmented_code)

from pywhy.tracer import get_tracer
_whyline_tracer = get_tracer()

def add(a, b):
    _whyline_tracer.record_event(6, '<string>', 2, 'function_entry', 'func_name', 'add', 'args', [a, b])
    if a < 10:
        _whyline_tracer.record_event(4, '<string>', 4, 'branch', 'taken', 'if')
        c = a + b
        _whyline_tracer.record_event(1, '<string>', 5, 'assign', 'var_name', 'c', 'value', c)
        _whyline_tracer.record_event(2, '<string>', 6, 'return', 'value', c)
        return c
    _whyline_tracer.record_event(5, '<string>', 8, 'return', 'value', b)
    return b
a = 5
_whyline_tracer.record_event(7, '<string>', 10, 'assign', 'var_name', 'a', 'value', a)
b = 10
_whyline_tracer.record_event(8, '<string>', 11, 'assign', 'var_name', 'b', 'value', b)
add(a, b)


In [3]:
globals_dict = exec_instrumented(source_code)
tracer = globals_dict["_whyline_tracer"]
tracer.get_stats()

{'total_events': 6,
 'event_types': {'assign': 3, 'function_entry': 1, 'branch': 1, 'return': 1},
 'files_traced': 1,
 'time_span': 8.606910705566406e-05}

In [4]:
for event in tracer.events:
    print(event)

TraceEvent(event_id=7, filename='<string>', lineno=10, event_type='assign', timestamp=1753189049.742646, thread_id=8272034496, locals_snapshot={'__builtins__': '<unpicklable: module>', '__name__': '__main__', '__file__': '<string>', 'get_tracer': <function get_tracer at 0x107639f80>, 'add': '<unpicklable: function>', 'a': 5}, globals_snapshot={'a': 5}, args=('var_name', 'a', 'value', 5), kwargs={})
TraceEvent(event_id=8, filename='<string>', lineno=11, event_type='assign', timestamp=1753189049.742702, thread_id=8272034496, locals_snapshot={'__builtins__': '<unpicklable: module>', '__name__': '__main__', '__file__': '<string>', 'get_tracer': <function get_tracer at 0x107639f80>, 'add': '<unpicklable: function>', 'a': 5, 'b': 10}, globals_snapshot={'a': 5, 'b': 10}, args=('var_name', 'b', 'value', 10), kwargs={})
TraceEvent(event_id=6, filename='<string>', lineno=2, event_type='function_entry', timestamp=1753189049.742717, thread_id=8272034496, locals_snapshot={'a': 5, 'b': 10}, globals_

# Whyline Questions

## Why was function x called?

- Find all traces where the function was called.
- For each of these traces go through previous events and find control flow events 
- For each control flow argument find relevant variables if nessiscarry to find 
- For each variable 

# Why did <method> return <return_value>?

# why didn’t field’s value change after time T

# why did object get created ?

# Why did <VARIABLE> get assigned <value>?
# Why did <property> get assigned <value>?

In [5]:
import ast
import sys
from typing import Dict, Set, List, Any

class DynamicSlicer:
    def __init__(self):
        self.execution_trace = []  # Store executed statements
        self.data_dependencies = {}  # Track variable dependencies
        self.control_dependencies = {}  # Track control flow dependencies
        self.variable_definitions = {}  # Track where variables are defined
        self.line_number = 0
        
    def trace_assignment(self, line_num: int, var_name: str, dependencies: Set[str]):
        """Record an assignment operation and its dependencies"""
        self.execution_trace.append({
            'type': 'assignment',
            'line': line_num,
            'variable': var_name,
            'dependencies': dependencies.copy()
        })
        
        # Update variable definition location
        self.variable_definitions[var_name] = line_num
        
        # Update data dependencies
        self.data_dependencies[var_name] = dependencies.copy()
        
    def trace_condition(self, line_num: int, condition_vars: Set[str], branch_taken: bool):
        """Record a conditional statement execution"""
        self.execution_trace.append({
            'type': 'condition',
            'line': line_num,
            'variables': condition_vars.copy(),
            'branch_taken': branch_taken
        })
        
    def get_dynamic_slice(self, target_var: str, target_line: int) -> Set[int]:
        """
        Compute the dynamic slice for a target variable at a specific line.
        Returns the set of line numbers that influence the target variable.
        """
        slice_lines = set()
        worklist = [(target_var, target_line)]
        processed = set()
        
        while worklist:
            var, line = worklist.pop(0)
            
            if (var, line) in processed:
                continue
            processed.add((var, line))
            
            # Find the most recent definition of this variable before the target line
            definition_line = None
            for trace_item in reversed(self.execution_trace):
                if (trace_item['type'] == 'assignment' and 
                    trace_item['variable'] == var and 
                    trace_item['line'] <= line):
                    definition_line = trace_item['line']
                    slice_lines.add(definition_line)
                    
                    # Add dependencies of this variable to worklist
                    for dep_var in trace_item['dependencies']:
                        worklist.append((dep_var, definition_line))
                    break
                    
            # Add control dependencies
            for trace_item in self.execution_trace:
                if (trace_item['type'] == 'condition' and 
                    trace_item['line'] < line):
                    # Check if this condition affects the target
                    if self._affects_control_flow(trace_item['line'], line):
                        slice_lines.add(trace_item['line'])
                        # Add variables used in condition
                        for cond_var in trace_item['variables']:
                            worklist.append((cond_var, trace_item['line']))
        
        return slice_lines
    
    def _affects_control_flow(self, condition_line: int, target_line: int) -> bool:
        """Simple heuristic to determine if a condition affects control flow to target"""
        # In a real implementation, this would use proper control flow analysis
        return condition_line < target_line

# Example program to demonstrate dynamic slicing
class TracedExecution:
    def __init__(self):
        self.slicer = DynamicSlicer()
        self.variables = {}
        
    def execute_example_program(self, x_input: int, y_input: int):
        """Execute an example program with tracing"""
        
        # Line 1: x = x_input
        line = 1
        self.variables['x'] = x_input
        self.slicer.trace_assignment(line, 'x', set())
        
        # Line 2: y = y_input  
        line = 2
        self.variables['y'] = y_input
        self.slicer.trace_assignment(line, 'y', set())
        
        # Line 3: z = 10
        line = 3
        self.variables['z'] = 10
        self.slicer.trace_assignment(line, 'z', set())
        
        # Line 4: if x > 5:
        line = 4
        condition = self.variables['x'] > 5
        self.slicer.trace_condition(line, {'x'}, condition)
        
        if condition:
            # Line 5: a = x + y
            line = 5
            self.variables['a'] = self.variables['x'] + self.variables['y']
            self.slicer.trace_assignment(line, 'a', {'x', 'y'})
            
            # Line 6: b = a * 2
            line = 6
            self.variables['b'] = self.variables['a'] * 2
            self.slicer.trace_assignment(line, 'b', {'a'})
        else:
            # Line 7: a = y - 1
            line = 7
            self.variables['a'] = self.variables['y'] - 1
            self.slicer.trace_assignment(line, 'a', {'y'})
            
            # Line 8: b = a + z
            line = 8
            self.variables['b'] = self.variables['a'] + self.variables['z']
            self.slicer.trace_assignment(line, 'b', {'a', 'z'})
        
        # Line 9: result = b + z
        line = 9
        self.variables['result'] = self.variables['b'] + self.variables['z']
        self.slicer.trace_assignment(line, 'result', {'b', 'z'})
        
        return self.variables['result']

# Demonstration
def demonstrate_dynamic_slicing():
    print("=== Dynamic Slicing Demonstration ===\n")
    
    # Test case 1: x > 5 (condition is true)
    print("Test Case 1: x=10, y=3 (x > 5 is True)")
    exec1 = TracedExecution()
    result1 = exec1.execute_example_program(10, 3)
    
    print("Program execution path:")
    for trace in exec1.slicer.execution_trace:
        print(f"  {trace}")
    
    print(f"\nFinal result: {result1}")
    
    # Get dynamic slice for 'result' at line 9
    slice_lines = exec1.slicer.get_dynamic_slice('result', 9)
    print(f"Dynamic slice for 'result' at line 9: {sorted(slice_lines)}")
    print("This means lines", sorted(slice_lines), "actually influenced the final result\n")
    
    # Test case 2: x <= 5 (condition is false)  
    print("Test Case 2: x=3, y=7 (x > 5 is False)")
    exec2 = TracedExecution()
    result2 = exec2.execute_example_program(3, 7)
    
    print("Program execution path:")
    for trace in exec2.slicer.execution_trace:
        print(f"  {trace}")
    
    print(f"\nFinal result: {result2}")
    
    slice_lines2 = exec2.slicer.get_dynamic_slice('result', 9)
    print(f"Dynamic slice for 'result' at line 9: {sorted(slice_lines2)}")
    print("This means lines", sorted(slice_lines2), "actually influenced the final result")
    
    print("\n=== Analysis ===")
    print("Notice how the dynamic slice is different for each execution:")
    print(f"- Case 1 (x=10): Uses lines {sorted(slice_lines)} - goes through the 'if' branch")
    print(f"- Case 2 (x=3): Uses lines {sorted(slice_lines2)} - goes through the 'else' branch")
    print("\nStatic slicing would include ALL possible lines, but dynamic slicing")
    print("only includes lines that actually executed and influenced the target variable.")

if __name__ == "__main__":
    demonstrate_dynamic_slicing()

=== Dynamic Slicing Demonstration ===

Test Case 1: x=10, y=3 (x > 5 is True)
Program execution path:
  {'type': 'assignment', 'line': 1, 'variable': 'x', 'dependencies': set()}
  {'type': 'assignment', 'line': 2, 'variable': 'y', 'dependencies': set()}
  {'type': 'assignment', 'line': 3, 'variable': 'z', 'dependencies': set()}
  {'type': 'condition', 'line': 4, 'variables': {'x'}, 'branch_taken': True}
  {'type': 'assignment', 'line': 5, 'variable': 'a', 'dependencies': {'x', 'y'}}
  {'type': 'assignment', 'line': 6, 'variable': 'b', 'dependencies': {'a'}}
  {'type': 'assignment', 'line': 9, 'variable': 'result', 'dependencies': {'z', 'b'}}

Final result: 36
Dynamic slice for 'result' at line 9: [1, 2, 3, 4, 5, 6, 9]
This means lines [1, 2, 3, 4, 5, 6, 9] actually influenced the final result

Test Case 2: x=3, y=7 (x > 5 is False)
Program execution path:
  {'type': 'assignment', 'line': 1, 'variable': 'x', 'dependencies': set()}
  {'type': 'assignment', 'line': 2, 'variable': 'y', 'de